In [31]:
import re 

split_words = [
    " de ",
    " d'"
]

unsplitable_words = [
    "huile d'olive",
    "herbes de provence"
]

prefix = "* "

units = [
    "g",
    'cl',
]

def p_u_word(ingredient: str):
    ingredient = ingredient.lower()
    for unsplitable_word in unsplitable_words:
        if unsplitable_word in ingredient.lower():
            si = " ".join(ingredient.split(unsplitable_word)).strip()
            if not si:
                return f"@{unsplitable_word}{{}}"
    return ""

def p_single_word(ingredient: str) -> str:
    if not " " in ingredient:
        return f"@{ingredient}"
    return ""

def p_s_word(ingredient: str):
    for word in split_words:
        if word in ingredient:
            split_ingredients = ingredient.split(word)
            ingredient_name = word.join(split_ingredients[1:]).strip()
            ingredient_q_u = split_ingredients[0].strip()
            formated_q_u = p_q_u(ingredient_q_u)
            return f"@{ingredient_name}{formated_q_u}"
    return ""

def p_q_u(q_u: str):
    """process quantity and unit
    """
    q_u_split = q_u.split(" ")
    if len(q_u_split) == 1:
        return f"{{{q_u}}}"
    q = q_u_split[0]
    u = " ".join(q_u_split[1:])
    return f"{{{q}%{u}}}"

def p_start_int(ingredient: str) -> str:
    i_split = ingredient.split(" ")
    if re.match(r"\d+", i_split[0]):
        if i_split[1] in units:
            ingred = " ".join(i_split[2:])
            return f"@{ingred}{{{i_split[0]}%{i_split[1]}}}"
        ingred = " ".join(i_split[1:])
        return f"@{ingred}{{{i_split[0]}}}"
    return ""

def process_ingredients(ingredients: str) -> str:
    out = ""
    for ingredient in ingredients.split("\n"):
        ingredient = ingredient.removeprefix(prefix).strip().lower()
        if not ingredient:
            continue
        if t := p_single_word(ingredient):
            out += t + "\n"
            continue
        if t := p_u_word(ingredient):
            out += t + "\n"
            continue
        if t := p_s_word(ingredient):
            out += t + "\n"
            continue
        if t := p_start_int(ingredient):
            out += t + "\n"
            continue
        out += f"@{ingredient}{{}}\n"
    return out

In [45]:
ingredients="""
* 250 g de tagliatelle
* 1 poireau
* 75 g de lardons
* 1/2 petit pot de crème fraîche
* Beurre
* Poivre
* Sel
* Parmesan
"""

protocol = """
* Laver puis couper les poireaux en fines rondelles,
* les mettre à fondre environ 15 min dans une poêle avec un bon morceau de beurre (remuer souvent).
* Mettre les tagliatelles à cuire comme indiqué sur le paquet.
* Attendre que les poireaux soient bien fondus et ajouter les lardons, faire dorer.
* Hors du feux, mettre la crème fraîche et remuer, poivrer et saler à votre goût.
* Égouter les tagliatelles, les mettre dans un plat et verser la sauce aux poireaux dessus.
* Ajouter du parmesan râpé.
* Servir aussitôt.
"""

In [46]:
out = process_ingredients(ingredients)
map_ingredient_to_cooklang = {
    l[1:].split("{")[0]: l for l in out.splitlines()
}
map_ingredient_to_cooklang

{'tagliatelle': '@tagliatelle{250%g}',
 'poireau': '@poireau{1}',
 'lardons': '@lardons{75%g}',
 'crème fraîche': '@crème fraîche{1/2%petit pot}',
 'beurre': '@beurre',
 'poivre': '@poivre',
 'sel': '@sel',
 'parmesan': '@parmesan'}

In [48]:
processed_ingredients = []
for step in protocol.split("\n"):
    step = step.strip().removeprefix("* ")
    if not step:
        continue
    for ingredient in map_ingredient_to_cooklang.keys():
        if ingredient.lower() in step and ingredient not in processed_ingredients:
            step = step.replace(ingredient, map_ingredient_to_cooklang[ingredient])
            processed_ingredients.append(ingredient)

    print(step)

print("\n".join(v for k,v in map_ingredient_to_cooklang.items() if k not in processed_ingredients))


Laver puis couper les @poireau{1}x en fines rondelles,
les mettre à fondre environ 15 min dans une poêle avec un bon morceau de @beurre (remuer souvent).
Mettre les @tagliatelle{250%g}s à cuire comme indiqué sur le paquet.
Attendre que les poireaux soient bien fondus et ajouter les @lardons{75%g}, faire dorer.
Hors du feux, mettre la @crème fraîche{1/2%petit pot} et remuer, @poivrer et saler à votre goût.
Égouter les tagliatelles, les mettre dans un plat et verser la sauce aux poireaux dessus.
Ajouter du @parmesan râpé.
Servir aussitôt.
@sel
